# KH-5 ARGON images final ortho-rectification

In [32]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
import pandas as pd
import json

# parallel computing
from dask.distributed import Client
# import dask.dataframe as dd

# personnal packages
import geometry.internal_orientation as gio
import geometry.external_orientation as geo

List of available KH-5 ARGON images:

In [2]:
images_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(images_root):
    if os.path.isdir(posixpath.join(images_root, x)):
        products.append(x)
        
products

['DS09034A007MC018',
 'DS09034A007MC019',
 'DS09034A008MC019',
 'DS09034A008MC020',
 'DS09034A008MC021',
 'DS09034A008MC022',
 'DS09058A024MC012',
 'DS09058A024MC013']

Read `.json` file containing all the internal orientation parameters:

In [11]:
all_params_path = posixpath.join(images_root, "images_params.json")
if os.path.exists(posixpath.join(images_root, "images_params.json")):
    with open(all_params_path, "r") as f:
        all_params = json.load(f)

Set up Dask client to prevent full rasters loading in memory:

In [5]:
def set_env():
    os.environ["GS_NO_SIGN_REQUEST"] = "YES"

set_env()

client = Client(n_workers=1, threads_per_worker=4)
client.run(set_env)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 7.89 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42543,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:42550,Total threads: 4
Dashboard: http://127.0.0.1:42551/status,Memory: 7.89 GiB
Nanny: tcp://127.0.0.1:42546,


## DS09058A024MC012

Load raster and extract shape

In [6]:
img_name = "DS09058A024MC012"
file = posixpath.join(images_root, img_name, img_name + "_a.tif")
raster = rioxarray.open_rasterio(file, chunks=True)
shx, shy = raster.shape[1:]
raster

c:\ProgramData\miniconda3\Lib\site-packages\rioxarray\_io.py:1143: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore


<xarray.DataArray (band: 1, y: 19912, x: 19332)> Size: 385MB
dask.array<open_rasterio-31a72a210d3b797622536cdec8c74bb8<this-array>, shape=(1, 19912, 19332), dtype=uint8, chunksize=(1, 6942, 19332), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int32 4B 1
  * x            (x) float64 155kB 0.5 1.5 2.5 ... 1.933e+04 1.933e+04 1.933e+04
  * y            (y) float64 159kB 0.5 1.5 2.5 ... 1.991e+04 1.991e+04 1.991e+04
    spatial_ref  int32 4B 0
Attributes: (12/13)
    TIFFTAG_SOFTWARE:          Adobe Photoshop CC 2018 (Windows)
    TIFFTAG_DATETIME:          2019:12:03 22:08:29
    TIFFTAG_XRESOLUTION:       1432.3507
    TIFFTAG_YRESOLUTION:       1432.3507
    TIFFTAG_RESOLUTIONUNIT:    3 (pixels/cm)
    STATISTICS_APPROXIMATE:    YES
    ...                        ...
    STATISTICS_MEAN:           118.38313472809
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         96.612358145086
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0

Create a mesh of simulated GCPs

In [35]:
lat_c, lon_c = all_params[img_name]["external_orientation"]["lat_c"], all_params[img_name]["external_orientation"]["lon_c"]

R = 6.357e6

d_lat = 2.5e5 / R * 180/np.pi
d_lon = 2.5e5 / (R * np.cos(lat_c*np.pi/180))* 180/np.pi
lat, lon, h = np.meshgrid(
        np.linspace(lat_c - d_lat, lat_c + d_lat, 100), 
        np.linspace(lon_c - d_lon, lon_c + d_lon, 100),
        np.arange(0, 2000, 100)
)
 
sim_GCPs = pd.DataFrame({
        "lat": lat.reshape(-1),
        "lon": lon.reshape(-1),
        "h": h.reshape(-1)
})

In [ ]:
# geocentric cartesian coordinates
x_geo, y_geo, z_geo = geo.geodetic_to_geocentric_cartesian_coordinates(sim_GCPs.lat.values *np.pi/180, sim_GCPs.lon.values *np.pi/180, sim_GCPs.h.values)
sim_GCPs.loc[:, ["x_geo", "y_geo", "z_geo"]] = np.array([x_geo, y_geo, z_geo]).T

# local cartesian coordinates
x_geo, y_geo, z_geo = sim_GCPs.loc[:, "x_geo"], sim_GCPs.loc[:, "y_geo"], sim_GCPs.loc[:, "z_geo"]
x_gr, y_gr, z_gr = geo.geocentric_cartesian_to_local_cartesian_coordinates(x_geo.values, y_geo.values, z_geo.values, lat_c *np.pi/180, lon_c *np.pi/180)
sim_GCPs.loc[:, ["x_gr", "y_gr", "z_gr"]] = np.array([x_gr, y_gr, z_gr]).T

# photo coordinates
x_gr, y_gr, z_gr = sim_GCPs.loc[:, "x_gr"], sim_GCPs.loc[:, "y_gr"], sim_GCPs.loc[:, "z_gr"]
xp, yp = geo.collinearity_equations(x_gr.values, y_gr.values, z_gr.values, 76.2e-3, 
                                      all_params[img_name]["internal_orientation"]["xc"], 
                                      all_params[img_name]["internal_orientation"]["yc"], 
                                      all_params[img_name]["external_orientation"]["zc"],
                                      all_params[img_name]["external_orientation"]["omega"] * np.pi / 180, 
                                      all_params[img_name]["external_orientation"]["phi"] * np.pi / 180, 
                                      all_params[img_name]["external_orientation"]["kappa"] * np.pi / 180)
sim_GCPs.loc[:, ["xp", "yp"]] = np.array([xp, yp]).T

# lens distortion ignored

# fiducial coordinates
xp, yp = sim_GCPs.loc[:, "xp"], sim_GCPs.loc[:, "yp"]
xi, eta = gio.photo_to_fiducial_coordinates(xp.values, yp.values, 
                                                all_params[img_name]["internal_orientation"]["x0"], 
                                                all_params[img_name]["internal_orientation"]["y0"])
sim_GCPs.loc[:, ["xi", "eta"]] = np.array([xi, eta]).T
    
# image coordinates
xi, eta = sim_GCPs.loc[:, "xi"], sim_GCPs.loc[:, "eta"]
x, y = gio.fiducial_to_image_coordinates(xi.values, eta.values, 
                                            all_params[img_name]["internal_orientation"]["xc"], 
                                            all_params[img_name]["internal_orientation"]["yc"], 
                                            all_params[img_name]["internal_orientation"]["alpha"] * np.pi / 180,
                                            all_params[img_name]["internal_orientation"]["delta_xi"],
                                            all_params[img_name]["internal_orientation"]["delta_eta"])
sim_GCPs.loc[:, ["x", "y"]] = np.array([x, y]).T
    
sim_GCPs.head()

,lat,lon,h,x_geo,y_geo,z_geo,x_gr,y_gr,z_gr,xi,eta,x,y,xp,yp
0,76.609145,1.772728,0,1.481122e+06,45840.443101,6.182813e+06,-299606.817137,-221467.251136,-10857.453781,-0.007114,0.015663,-1082.958507,2455.736673,-0.070614,-0.047837
1,76.609145,1.772728,100,1.481145e+06,45841.159533,6.182910e+06,-299611.499637,-221470.713495,-10757.623494,-0.007137,0.015648,-1086.351337,2453.369232,-0.070637,-0.047852
2,76.609145,1.772728,200,1.481168e+06,45841.875965,6.183008e+06,-299616.182137,-221474.175853,-10657.793206,-0.007159,0.015633,-1089.746203,2451.000370,-0.070659,-0.047867
3,76.609145,1.772728,300,1.481191e+06,45842.592397,6.183105e+06,-299620.864637,-221477.638211,-10557.962919,-0.007181,0.015618,-1093.143108,2448.630086,-0.070681,-0.047882
4,76.609145,1.772728,400,1.481214e+06,45843.308828,6.183202e+06,-299625.547137,-221481.100569,-10458.132632,-0.007204,0.015602,-1096.542052,2446.258378,-0.070704,-0.047898
